<a href="https://colab.research.google.com/github/mariamingallonMM/AI-ML-W11-Probabilistic-Matrix-Factorization/blob/main/ai_columbiax_ml_w11_pmf_codelab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction
This notebook demonstrates the use of probabilistic matrix factorization (PMF) model which works as follows:

* fills in the values of a missing matrix M,
* where Mij is an observed value if (i,j)∈Ω and
* where Ω contains the measured pairs.

The goal of PMF is to factorize the M matrix into a product between vectors such that Mij ≈ uTi vj, where each ui,vj∈Rd. The modeling problem is to learn ui for i=1,…,Nu and vj for j=1,…,Nv by maximizing the objective function, which can be formulated as follows:

L=−∑(i,j)∈Ω12σ2(Mij−uTivj)2−∑Nui=1λ2∥ui∥2−∑Nvj=1λ2∥vj∥2]

For this assignment we are asked to set d, sigma and lambda to a particular value as follows:

d=5, dimensions of the rank
σ2=1/10, covariance of Gaussian distribution
λ=2, lambda of Gaussian distribution
It is also available on GitHub at: https://github.com/mariamingallonMM/AI-ML-W11-Probabilistic-Matrix-Factorization

#How it works
Probabilistic Matrix Factorization (PMF) is used commonly for collaborative filtering. The latter is used as an alternative to content-based filtering when there is not enough information provided by a user to make suggestions. While content-based filtering makes use of the user's explicitly expressed preferences, collaborative filtering** uses the history data provided by a group of users with similar preferences to make elicit recommendations.

Whereas in content-based filering we expect for a given user to build a profile that clearly states preferences, in collaborative-filtering this information may not be fully available, but we expect our system to still be able to make recommendations based on evidence that similar users provide.

The following is how we have implemented Probabilistic Matrix Factorization for building a movie recommendation system using collaborative filtering:

Transform input ratings.csv to M matrix, of n rows and m columns, where each row is a user and each column is a movie. Where we don't have data, we will use a '0' instead of NaN. Users and movies shall be indexed from 1 (not '0').
We estimate the M matrix by using two low-rank matrices U and V as: M = UT x V, where: a. UT is the transposed matrix of U. UT is an n x d matrix, where n is the number of users (rows of M), and d is the rank (d fixed to 5 in this assignment). b. V is a d x m matrix, where m is the number of movies to rate (columns in M).

We will use MAP inference coordinate ascent algorithm to estimate the missing ratings of 5 users for 5 movies not already rated in the starting dataset.
First, we will initialize each vj with a normal distribution of zero mean and covariance equal to the inverse of lambda multiplied by the identity matrix.
For each iteration, we update ui and then vj.

The PMF algorithm must learn 5 dimensions (d=5) and shall be run for 50 iterations.

In [14]:
import sys
import os
import math
from random import randrange
import functools
import operator
import requests
import psutil

# 3rd party modules
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp
from scipy.cluster.vq import kmeans2
from scipy.stats import multivariate_normal
from scipy.spatial.distance import cdist
from scipy.special import logsumexp
from scipy import stats

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

from IPython.display import HTML

##Step 1: Import the dataset
The input csv file ('ratings.csv') is a comma separated file containing three columns: user_index, object_index, and rating.

In [15]:
def get_data(filename, **kwargs):
    """
    Read data from a file given its name. Option to provide the path to the file if different from: [./datasets/in].
    ------------
    Parameters:
    - filename: name of the file to be read, to get the data from.
    - kwargs (optional): 
        - 'headers': list of str for the headers to include in the outputs file created
        - 'path': str of the path to where the file is read, specified if different from default ([./datasets/in])
    ------------
    Returns:
    - df: a dataframe of the data
    - users: list of the users ids
    - objects: list of the objects ids

    """
    
    # Define input filepath
    if 'path' in kwargs:
        filepath = kwargs['path']
    else:
        filepath = os.path.join(os.getcwd(),'datasets','out')

    input_path = os.path.join(filepath, filename)

    # If provided, use the title of the columns in the input dataset as headers of the dataframe
    if 'headers' in kwargs:
        # Read input data
        df = pd.read_csv(input_path, names = kwargs['headers'])
    else:
        # Read input data
        df = pd.read_csv(input_path)
       
    return df

Call the function get_data to read the dataset



In [16]:

train_data = get_data('ratings_sample.csv', path = "/content/input/movieratings/datasets/", headers=['user_id', 'movie_id', 'rating'])
train_data.head()

,user_id,movie_id,rating
0,1.0,181.0,-0.249054
1,1.0,353.0,0.390805
2,1.0,136.0,-1.674198
3,1.0,277.0,-0.226196
4,1.0,187.0,1.193406


##Define main PMF function¶


In [17]:
def PMF(train_data, headers = ['user_id', 'movie_id'], lam:int = 2, sigma2:float = 0.1, d:int = 5, iterations:int = 50, output_iterations:list=[10,25,50]):
    """
    Implements Probabilistic Matrix Factorization.

    ------------
    Parameters:

    - data: dataset used for training (e.g. the ratings.csv dataset with missing values for users and movies).
    - headers: title of the headers in the dataset for the 'users id' and 'movie id' values.
    - lam: lambda value to initialise the Gaussian zero mean distribution (default lam = 2 for this assignment).
    - sigma2: covariance of the Gaussian (default sigma2 = 0.1 for this assignment).
    - d: number of dimensions for the ranking, (default d = 5 for this assignment).
    - iterations: number of iterations to run PMF for (default, 50 iterations).
    
    ------------
    Returns:

    - L: Loss
    - U_matrices: matrices of users
    - V_matrices: matrices of objects

    """

    L_results = []
    U_matrices = {}
    V_matrices = {}
    log_aps = []

    # first convert dataframe to the ratings matrix as a sparse matrix
    M, n, m, users, objects, rows, cols = df_to_ratings_matrix(train_data, headers = headers)

    parameters = initialize_parameters(lam, n, m, d)

    for i in range(1, iterations + 1):
        new_parameters = update_parameters(M, parameters, lam, n, m, d)
        log_ap = log_a_posteriori(M, parameters)
        L_results.append(log_ap)

        if i in output_iterations:
            print('Log p a-posteriori at iteration ', i, ':', log_ap)
            U_matrices[i] = new_parameters['U']
            V_matrices[i] = new_parameters['V']

    return L_results, U_matrices, V_matrices, users, objects, new_parameters, M, rows, cols

##Define helpers of PMF main function¶
Initialize our parameters (U, V, lambda_U and lambda_V).




In [18]:
def initialize_parameters(lam, n, m, d):
    """
    Initializes our parameters. First the V matrix as a random Gaussian zero mean distribution from a given lambda.
    
    ------------
    Parameters:

    - lam: lambda value to initialise the Gaussian zero mean distribution (default lam = 2 for this assignment).
    - n: number of users in dataset
    - m: number of movies in dataset
    - d: number of dimensions for the ranking, (default d = 5 for this assignment).

    ------------
    Returns:
    
    - parameters: a dictionary with the values for: 
        - U: matrix of users
        - V: matrix of objects (movies in this case)
        - lambda_U: value of lambda, per the inputs
        - lambda_V: value of lambda, per the inputs

    """

    U = np.zeros((d, n), dtype=np.float64)
    V = np.random.normal(0.0, 1.0 / lam, (d, m))
    
    parameters = {}

    parameters['U'] = U
    parameters['V'] = V
    parameters['lambda_U'] = lam
    parameters['lambda_V'] = lam
    
    return parameters

Updates the parameters U and V while the iterative PMF function is running

In [19]:
def update_parameters(M, parameters, lam, n, m, d):
    """
    Implements the function that updates U and V.

    ------------
    Parameters:
    
    - M: the ratings matrix, as sparse (zeros used to fill the nan, missing values)   
    - parameters: a dictionary with the values for: 
        - U: matrix of users
        - V: matrix of objects (movies in this case)
        - lambda_U: value of lambda, per the inputs
        - lambda_V: value of lambda, per the inputs    
    - lam: lambda value to initialise the Gaussian zero mean distribution (default lam = 2 for this assignment).
    - n: number of users in dataset
    - m: number of movies in dataset
    - d: number of dimensions for the ranking, (default d = 5 for this assignment).
    
    ------------
    Returns:

    - parameters: a dictionary with the values for: 
        - U: matrix of users
        - V: matrix of objects (movies in this case)
        - lambda_U: value of lambda, per the inputs
        - lambda_V: value of lambda, per the inputs

    """

    U = parameters['U']
    V = parameters['V']
    lambda_U = parameters['lambda_U']
    lambda_V = parameters['lambda_V']

    
    for i in range(n):
        V_j = V[:, M[i, :] > 0]
        U[:, i] = np.dot(np.linalg.inv(np.dot(V_j, V_j.T) + lambda_U * np.identity(d)), np.dot(M[i, M[i, :] > 0], V_j.T))
        
    for j in range(m):
        U_i = U[:, M[:, j] > 0]
        V[:, j] = np.dot(np.linalg.inv(np.dot(U_i, U_i.T) + lambda_V * np.identity(d)), np.dot(M[M[:, j] > 0, j], U_i.T))
        
    parameters['U'] = U
    parameters['V'] = V

    min_rating = np.min(M)
    max_rating = np.max(M)

    return parameters

Calculate the log-a posteriori (L)



In [20]:
def log_a_posteriori(M, parameters):
    """
    Implements the Log-a posteriori with equation as follows:
    
    L=-\frac 1 2 \left(\sum_{i=1}^N\sum_{j=1}^M(R_{ij}-U_i^TV_j)_{(i,j) \in \Omega_{R_{ij}}}^2+\lambda_U\sum_{i=1}^N\|U_i\|_{Fro}^2+\lambda_V\sum_{j=1}^M\|V_j\|_{Fro}^2\right)

    ------------
    Parameters:
    
    - M: the ratings matrix, as sparse (zeros used to fill the nan, missing values)
    - parameters: a dictionary with the values for: 
        - U: matrix of users
        - V: matrix of objects (movies in this case)
        - lambda_U: value of lambda, per the inputs
        - lambda_V: value of lambda, per the inputs
        
    ------------
    Returns:
    
    - L: the resulting float number from the above equation of 'L'

    """

    lambda_U = parameters['lambda_U']
    lambda_V = parameters['lambda_V']
    U = parameters['U']
    V = parameters['V']
    
    UV = np.dot(U.T, V)
    M_UV = (M[M > 0] - UV[M > 0])
    
    L = -0.5 * (np.sum(np.dot(M_UV, M_UV.T)) + lambda_U * np.sum(np.dot(U, U.T)) + lambda_V * np.sum(np.dot(V, V.T)))
    
    return L

In [21]:
def save_outputs_txt(data, output_iterations:list = [5, 10, 25]):
    """
    Write the outputs to csv files.

    ------------
    Parameters:

    - data: a list of the resulting matrixes to write as outputs.
    - output_iterations: the iterations to store as output csv files for the U and V matrixes.
    
    ------------
    Returns:

    - csv files with the output data

    """

    L_results = data[0]
    np.savetxt("objective.csv", L_results, delimiter=",")

    U_results = data[1]
    V_results = data[2]

    for i in output_iterations:
        filename = "U-" + str(i) + ".csv"
        np.savetxt(filename, U_results[i].T, delimiter=",")
        filename = "V-" + str(i) + ".csv"
        np.savetxt(filename, V_results[i].T, delimiter=",")

    return

In [22]:
def df_to_ratings_matrix(df, **kwargs):
    """
    Converts a given dataframe to a sparse matrix, in this case the M ratings matrix.

    ------------
    Parameters:

    - df: dataframe used for training (e.g. the ratings.csv dataset with missing values for users and movies).
    - headers (optional): title of the headers in the dataset for the 'users id' and 'movie id' values.
   
    ------------
    Returns:

    - M: the ratings matrix, as sparse (zeros used to fill the nan, missing values)
    - n: number of rows
    - m: number of columns
    - users: list of unique users
    - movies: list of unique movies
    - rows: rows of the matrix M
    - cols: columns of the matrix M
    
    """

    df = df.dropna(how='all')

    if 'headers' in kwargs:
        headers = kwargs['headers']
        users_header = headers[0]
        movies_header = headers[1]
    else:
        users_header = 'user_id'
        movies_header = 'movie_id'


    users = df[users_header].unique()
    movies = df[movies_header].unique()
    df_values = df.values
   
    # initialise M ratings matrix as a sparse matrix of zeros
    M = np.zeros((len(users), len(movies)))

    rows = {}
    cols = {}

    for i, user_id in enumerate(users):
        rows[user_id] = i

    for j, movie_id in enumerate(movies):
        cols[movie_id] = j
    
    for index, row in df.iterrows():
        i = rows[row.user_id]
        j = cols[row.movie_id]
        M[i, j] = row.rating
    
    n = len(users) #number of rows
    m = len(movies) #number of columns
    
    return M, n, m, users, movies, rows, cols

#Predictions
Define the prediction function and the function to obtain a dataframe with the prediction

In [23]:
def predict(M, rows, cols, parameters, user_id, movie_id):
    """
    Predicts the rating value. Note the value has been scaled within the range 0-5.

    ------------
    Parameters:

    - M: the ratings matrix, as sparse (zeros used to fill the nan, missing values)   
    - rows: rows of the matrix M
    - cols: columns of the matrix M
    - parameters: a dictionary with the values for: 
        - U: matrix of users
        - V: matrix of objects (movies in this case)
        - lambda_U: value of lambda, per the inputs
        - lambda_V: value of lambda, per the inputs
    - user_id: id of the users being examined
    - movie_id: id of the objects being rated
   
    ------------
    Returns:
    
    - rating: a float number of the predicted rating for the object and user pair

    """

    U = parameters['U']
    V = parameters['V']
    
    M_ij = U[:, rows[user_id]].T.reshape(1, -1) @ V[:, cols[movie_id]].reshape(-1, 1)

    min_rating = np.min(M)
    max_rating = np.max(M)

    return 0 if max_rating == min_rating else ((M_ij[0][0] - min_rating) / (max_rating - min_rating)) * 5.0


def get_prediction(user_id, movies, M, rows, cols, parameters):
    """
    Obtain a dataframe of users Ids, movies Ids and the predicted rating for a given user Id.
    
    ------------
    Parameters:

    - user_id: the id of the user being examined
    - movies: the list of unique movie Ids
    - M: the ratings matrix, as sparse (zeros used to fill the nan, missing values)   
    - rows: rows of the matrix M
    - cols: columns of the matrix M
    - parameters: a dictionary with the values for: 
        - U: matrix of users
        - V: matrix of objects (movies in this case)
        - lambda_U: value of lambda, per the inputs
        - lambda_V: value of lambda, per the inputs

    ------------
    Returns:
    
    - df_result: a dataframe of users Ids, movies Ids and the predicted rating for a given user Id
    
    """

    predictions = np.zeros((len(movies), 1))
    df_result = pd.DataFrame(columns=['UserID', 'MovieID', 'Prediction'])

    for i, movie_id in enumerate(movies):
        predictions[i] = predict(M, rows, cols, new_parameters, user_id, movie_id)
        df_row = pd.DataFrame({
            'UserID': user_id,
            'MovieID': movie_id,
            'Prediction': predictions[i]
            })
        df_result = df_result.append(df_row, sort=False)
    
    return df_result

Execute the PMF function¶


In [24]:
out_iterations = [10, 25, 50]

# Assuming the PMF function returns Loss L, U_matrices and V_matrices
L_results, U_matrices, V_matrices, users, movies, new_parameters, M, rows, cols = PMF(train_data, headers = ['user_id', 'movie_id'], lam = 2, sigma2 = 0.1, d = 5, iterations = 50, output_iterations = out_iterations)

Log p a-posteriori at iteration  10 : -114.9654576784622
Log p a-posteriori at iteration  25 : -104.47858350032692
Log p a-posteriori at iteration  50 : -99.99884788741527


In [25]:
df_results = get_prediction(user_id = 5, movies = movies, M = M, rows = rows, cols = cols, parameters = new_parameters)
df_results.head()

,UserID,MovieID,Prediction
0,5,181.0,2.384809
0,5,353.0,2.456096
0,5,136.0,2.384809
0,5,277.0,2.384809
0,5,187.0,2.602499


#Notes on data repositories
We are using the ratings_sample.csv dataset provided with the assignment.
#Citations & References
PMF for Recommender Systems by Oscar Contreras Carrasco